In [22]:
#Please comment your code clearly, as the clarity of your code will also
# be evaluated. 

# 4.1

 # currently some params aren't moving off from initial guess.
 
 # unlike in YG's example, no $\Pi$

In [94]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import statsmodels.api as sm
import statsmodels.sandbox.regression.gmm as gmm
from statsmodels.sandbox.regression.gmm import IV2SLS
##############################################################################
# 1. Read Data & Construct Outside Good
##############################################################################
def read_data_with_outside_good(csv_path):
    """
    Reads a dataset that has columns:
        market, choice, shares, p, x, z1, z2, ...
    If the outside good (choice=0) is *not* in the CSV, this function
    creates a synthetic row for each market with p0=x0=0, share=1-sum(shares_in_market),
    and zeros for instruments if needed.
    """
    print(">>> Reading CSV and ensuring each market has an outside good row...")
    df = pd.read_csv(csv_path).copy()
    df.sort_values(by=["market", "choice"], inplace=True)
    df.reset_index(drop=True, inplace=True)

    frames = []
    for m in df["market"].unique():
        this_mkt = df.loc[df["market"] == m].copy()
        outside_row = this_mkt.loc[this_mkt["choice"] == 0]

        if outside_row.empty:
            sum_inside = this_mkt["shares"].sum()
            outside_share = max(0.0, 1.0 - sum_inside)
            row_outside = {
                "market": m,
                "choice": 0,
                "shares": outside_share,
                "p": 0.0,
                "x": 0.0,
            }
            # For z1,z2,... set to 0
            for c in this_mkt.columns:
                if c.startswith("z"):
                    row_outside[c] = 0.0
            this_mkt = pd.concat([this_mkt, pd.DataFrame([row_outside])],
                                 ignore_index=True)

        this_mkt.sort_values("choice", inplace=True)
        frames.append(this_mkt)

    df_out = pd.concat(frames, ignore_index=True)
    df_out.sort_values(by=["market", "choice"], inplace=True)
    df_out.reset_index(drop=True, inplace=True)
    return df_out


##############################################################################
# 2. Draw (vi) ~ N(0, I)
##############################################################################
def draw_taste_shocks(n_draws=50, seed=0):
    """
    Return an array of shape (n_draws, 2) of normal(0,1) draws
    for random coefficients dimension=2 (e.g., for price & x).
    """
    rng = np.random.default_rng(seed)
    v = rng.normal(0.0, 1.0, size=(n_draws, 2))
    return v


##############################################################################
# 3. Predict Shares (Vectorized)
##############################################################################
def predict_shares(delta, X, Gamma, v_draws, by_market):  # this is the function that implements YG step 2.
    """
    Vectorized share prediction:
      s_j = (1/n_draws) sum_{i=1..n_draws} exp( delta_j + X_j @ (Gamma v_i) )
                                        / sum_{k} exp( delta_k + X_k @ (Gamma v_i) ),
    by market.  We fix delta for the outside good = 0 in the contraction code.
    """
    n_draws = v_draws.shape[0]
    s_pred = np.zeros_like(delta)

    # Precompute random coefficients for all draws => shape (n_draws, 2)
    # That is rc[i] = Gamma @ v_draws[i]
    rc = (Gamma @ v_draws.T).T  # shape (n_draws, 2) #Sigma is Gamma in YG's notatation

    unique_mkts = np.unique(by_market) # not the nicest, but I for-loop over markets to calculate shares. might be a faster way to do this.
    for m_id in unique_mkts:
        idx = (by_market == m_id) # not understanding why I did this, since it seems to set idx to a boolean. TO DO: figure that out.

        delta_m = delta[idx]          # (#products_in_market,)
        X_m     = X[idx,:]            # (#products_in_market, 2)

        # mu_{j,i} = X_j dot rc_i => shape (#products_in_market, n_draws)
        mu = X_m @ rc.T
        util = delta_m[:, None] + mu  # broadcast => (#products_in_market, n_draws)
        exp_util = np.exp(util)

        denom = exp_util.sum(axis=0)  # sum over j => shape (n_draws,)
        s_pred[idx] = (exp_util / denom).mean(axis=1)

    return s_pred


##############################################################################
# 4. Contraction Mapping with Progress Prints
##############################################################################
def contraction_mapping(s_obs, X, Gamma, v_draws, by_market,
                       tol=1e-6, max_iter=100000, print_every=500):
    """
    Solves for delta_j subject to s_pred(delta) = s_obs.
    Outside good => delta_0=0. We only update inside goods in the iteration.
    """
    print(">>> Starting contraction mapping to solve for delta_j...")
    outside_idx = np.all(np.isclose(X, 0.0), axis=1) #index of outside good.
    inside_idx  = ~outside_idx # all other indices

    s_obs_safe = s_obs.clip(1e-16) # assure no 0 values.
    delta = np.log(s_obs_safe) 
    delta[outside_idx] = 0.0 #the delta_j for the outside good must be 0 ### TO DO: Do I need to include this? Yes, slide 2 of YG. 

    for iteration in range(max_iter): # this deviates from YG's code in that I do stop eventually
        # Periodically print progress
        if iteration % print_every == 0 and iteration > 0:
            pct = 100.0 * iteration / max_iter # pct complete with iterations
            print(f"    Contraction mapping iteration: {iteration}/{max_iter} "
                  f"({pct:.1f}% done)")

        s_pred = predict_shares(delta, X, Gamma, v_draws, by_market) # this is  YG step 2. 
        s_pred_safe = s_pred.clip(1e-16) # imposes a strictly positive floor so we can take logs. 

        delta_new = delta.copy()  # this is creating delta_k+1
        delta_new[inside_idx] = (  # YG step 3. 
            delta[inside_idx]
            + np.log(s_obs_safe[inside_idx])
            - np.log(s_pred_safe[inside_idx])
        )
        delta_new[outside_idx] = 0.0 # continues to impose outside option values are 0.

        sup_norm = np.max(np.abs(delta_new - delta))
        delta = delta_new
        if sup_norm < tol:
            print(f">>> Contraction mapping converged after {iteration+1} iterations.")
            break

    return delta


##############################################################################
# 5a. GMM Objective
##############################################################################
def gmm_objective(params, s_obs, X, Z, v_draws, by_market, W):
    """
    Objective = (Z' xi)' W (Z' xi), where xi = delta - X beta,
    and delta is solved from the contraction mapping.
    """
   
    gamma11, gamma21, gamma22 = params

    Gamma = np.array([
        [gamma11, 0.0],
        [gamma21, gamma22]
    ])
    # Solve for delta given parameters.
    delta = contraction_mapping(s_obs, X, Gamma, v_draws, by_market,
                                print_every=9999999)  # large so no spam
    Z_with_const = sm.add_constant(Z, has_constant='add')  # Shape: (N, 7)

    # IV2SLS Model  (see YG slide 11)
    iv_model = IV2SLS(endog=delta, exog=X, instrument=Z_with_const)
    iv_results = iv_model.fit()
    beta =  iv_results.params # slope on p, x.

    xi = delta - X @ beta
    m = Z.T @ xi
    obj = m.T @ W @ m
    print("obj")
    print(obj)
    print("beta")
    print(beta)
    print("gamma")
    display(Gamma)
    return obj


def approximate_delta_grad(
        gamma, 
        s_obs, 
        X, 
        v_draws, 
        by_market,
        h=1e-5
    ):
    """
    Numerically approximate d(delta)/dGamma at the current Gamma.
    
    gamma = [gamma11, gamma21, gamma22]
    Returns: ddelta_dgamma, shape (N, 3)
    """
    # 1) Evaluate baseline
    gamma11, gamma21, gamma22 = gamma
    Gamma_base = np.array([[gamma11, 0.0],
                           [gamma21, gamma22]])
    delta_base = contraction_mapping(s_obs, X, Gamma_base, v_draws, by_market)

    n = len(delta_base)
    ddelta_dgamma = np.zeros((n, 3))

    # 2) For each dimension j in gamma
    for j in range(3):
        step = np.zeros(3)
        step[j] = h  # small perturb

        Gamma_pert = np.array([
            [gamma11 + step[0], 0.0],
            [gamma21 + step[1], gamma22 + step[2]]
        ])
        delta_pert = contraction_mapping(s_obs, X, Gamma_pert, v_draws, by_market)
        ddelta_dgamma[:, j] = (delta_pert - delta_base) / (step[j])
    
    return ddelta_dgamma

def compute_2sls_beta(delta, X, Z):
    """
    Compute the 2SLS coefficient vector (beta) from the linear model:
       delta = X beta + error
    with instruments Z (including a constant column).
    
    2SLS formula: beta = (X' P_Z X)^(-1) X' P_Z delta
    where P_Z = Z (Z'Z)^(-1) Z'
    """
    # Projection of Z:
    iv_model = gmm.IV2SLS(endog=delta, exog=X, instrument=Z)
    results = iv_model.fit()

    # statsmodels returns all results in 'results.params'
    # This includes the intercept if it was in `X`.
    beta_2sls = results.params
   
    return beta_2sls

def gmm_objective_and_grad(
        gamma,
        s_obs,
        X,
        Z,
        v_draws,
        by_market,
        W
    ):
    """
    Returns: (objective, gradient wrt gamma),
    where gamma = [gamma11, gamma21, gamma22].
    
    Objective = (Z' xi)' W (Z' xi),
    with xi = delta - X beta_2sls,
         delta = contraction_mapping(...).
    """
    # 1) Build Gamma, solve for delta
    gamma11, gamma21, gamma22 = gamma
    Gamma = np.array([[gamma11, 0.0],
                      [gamma21, gamma22]])
    delta = contraction_mapping(s_obs, X, Gamma, v_draws, by_market)

    # 2) Insert a constant column into Z if not already:
    #    (or you can do sm.add_constant(Z) just once outside.)
    #    We'll do it manually here:
    ones = np.ones((Z.shape[0], 1))
    Z_with_const = np.hstack([ones, Z])

    # 3) Compute 2SLS beta
    beta_2sls = compute_2sls_beta(delta, X, Z_with_const)

    # 4) xi = delta - X beta
    xi = delta - X @ beta_2sls

    # 5) Evaluate objective
    #    m = Z' xi
    m = Z.T @ xi
    obj = m.T @ W @ m

    # ============= Gradient Computation =============
    #(NOT SURE IF THIS IS ALL EXACTLY CORRECT?)
    # 6) We need d(delta)/d(gamma):
    ddelta_dgamma = approximate_delta_grad(gamma, s_obs, X, v_draws, by_market)

    # 7) Then d(beta_2sls)/d(gamma):
    #    beta = B delta, B = (X'PZ X)^(-1) X'PZ
    #    so d beta / d gamma = B d delta / d gamma
    #    But let's do it "inside" the chain rule for xi.
    ZtZ_inv = np.linalg.inv(Z_with_const.T @ Z_with_const)
    PZc = Z_with_const @ ZtZ_inv @ Z_with_const.T
    XPZX = X.T @ PZc @ X
    XPZX_inv = np.linalg.inv(XPZX)
    B = XPZX_inv @ (X.T @ PZc)

    # 8) d(xi)/d(gamma) = d(delta)/d(gamma) - X * d(beta)/d(gamma)
    #                   = [I - X B] d(delta)/d(gamma)
    #    So define M = I - X B
    n = len(delta)
    M = np.eye(n) - X @ B  # shape (N, N)

    # 9) derivative wrt xi of the objective:
    #    obj = (Z' xi)^T W (Z' xi)
    #    => g_xi = 2 * Z W (Z' xi)
    g_xi = 2.0 * (Z @ (W @ (Z.T @ xi)))  # shape (N,)

    # 10) gradient wrt gamma is g_xi^T * [ M * ddelta_dgamma ]
    #     i.e. for each col j of ddelta_dgamma, we do g_xi^T @ (M @ col_j)
    #     That yields one number per j => gradient shape (3,)
    ddelta = M @ ddelta_dgamma  # shape (N,3)
    grad_gamma = ddelta.T @ g_xi  # shape (3,)
    print("obj")
    print(obj)
    print("beta")
    print(beta_2sls)
    print("gamma")
    print(Gamma)
    return obj, grad_gamma
##############################################################################
# 6. Two-Step GMM with Progress Messages
##############################################################################
def estimate_blp_iterative_gmm(df, n_draws=50, method="BFGS", use_analytic_grad=True):
    """
    1) Build Z, initial W1
    2) Minimize GMM => res1
    3) Recompute W => W2
    4) Minimize => res2
    """
    df.sort_values(by=["market","choice"], inplace=True) # sorts by market, then by choice
    df.reset_index(drop=True, inplace=True) # reset index

    s_obs = df["shares"].values # get shares
    X = df[["p","x"]].values #X is all features that we observe basically, price, and 'x' (quality)
    z_cols = [c for c in df.columns if c.startswith("z")] #instruments
    if not z_cols:
        raise ValueError("No instrument columns found!")
    Z = df[z_cols].values #instruments

    by_market = df["market"].values # markets
    # we do GMM 
    # weighting matrix
    print(">>> Building first-stage weighting matrix W1 = (Z'Z)^-1 ...")
    ZZ = Z.T @ Z
    eye_k = np.eye(ZZ.shape[0])
    W1 = np.linalg.inv(ZZ + 1e-12 * eye_k) # adding this ensures invertibility

    # Draw taste shocks
    v_draws = draw_taste_shocks(n_draws=n_draws, seed=0)


    # 3 parameters.  Lower triangle of gamma (3),
    init_params = np.array([1,1,1])
    print(">>> Starting FIRST pass GMM optimization...")


    # Use initial guess.
    def obj_grad_fun_1(par):
        return gmm_objective_and_grad(
            par,
            s_obs,
            X,
            Z,
            v_draws,
            by_market,
            W1
        )

    def obj_grad_fun_1(par):
        return gmm_objective_and_grad(par, s_obs, X, Z, v_draws, by_market, W1)

    # Use analytic gradient or not.
    if use_analytic_grad:
        
        res1 = minimize(
            obj_grad_fun_1,
            init_params,
            method=method,
            jac=True,     # Tells SciPy we are returning both (obj, grad).
            options={"disp": True, "maxiter": 15})
    # Note, this 'minimize' means essentially that we will optimize over params
    # This is Inner Loop step 2 and 3 in YG example
    else:
        res1 = minimize(obj_fun_1,
                        init_params,
                        method=method,
                        options={"disp": True, "gtol":1e-8})

    print(f"    [FIRST PASS] success={res1.success}, params={res1.x}, obj={res1.fun:.6g}")



    return res1


##############################################################################
# 7. Main 
##############################################################################
if __name__ == "__main__":
    # 1) Read data, ensuring outside good rows
    df_all = read_data_with_outside_good("ps1_ex4.csv")

    # 2)  GMM with partial grad and progress prints
    first_pass = estimate_blp_iterative_gmm(df_all,
                                                         n_draws=100,
                                                         method="BFGS",
                                                         use_analytic_grad=True)

    print("\n=== FIRST PASS RESULTS ===")
    print("Optimization success?", first_pass.success)
    print("Params:", first_pass.x)
    print("Objective:", first_pass.fun)



>>> Reading CSV and ensuring each market has an outside good row...
>>> Building first-stage weighting matrix W1 = (Z'Z)^-1 ...
>>> Starting FIRST pass GMM optimization...
>>> Starting contraction mapping to solve for delta_j...
>>> Contraction mapping converged after 364 iterations.
>>> Starting contraction mapping to solve for delta_j...
>>> Contraction mapping converged after 364 iterations.
>>> Starting contraction mapping to solve for delta_j...
>>> Contraction mapping converged after 364 iterations.
>>> Starting contraction mapping to solve for delta_j...
>>> Contraction mapping converged after 364 iterations.
>>> Starting contraction mapping to solve for delta_j...
>>> Contraction mapping converged after 364 iterations.
obj
878.6204573292839
beta
[-0.4386537  -2.27102483]
gamma
[[1. 0.]
 [1. 1.]]
>>> Starting contraction mapping to solve for delta_j...
>>> Contraction mapping converged after 322 iterations.
>>> Starting contraction mapping to solve for delta_j...
>>> Contraction

KeyboardInterrupt: 

In [ ]:
Gamma = np.array([[6.13440981 * 10**0, 0.00000000e+00],
                  [-1.33296619 * 10**(-1), 2.85100016 * 10**(-3)]])

# Multiply Gamma by its transpose
Omega = np.dot(Gamma, Gamma.T)

array([[ 3.76309837e+01, -8.17696087e-01],
       [-8.17696087e-01,  1.77761168e-02]])

#4.2    
 For each market, compute cross and own product elasticities. Average your results across
markets and present them in a J ×J table whose (i, j) element contains the (average) elasticity
of product i with respect to an increase in the price of product j. What’s the main difference
when compared with the table of elasticities you found in 2.2?


see slide 14 of Yonggeun's slides. we need to use the prices, shares, alpha_i (not clear how that maps to here) , and density of q_i (not sure how that maps to here), and density of v_i (assumed above)

\begin{table}
\caption{Average Price, x, and Share by Product}
\label{tab:avg_price_x_share}
\begin{tabular}{rrrr}
\toprule
choice & avg_price & avg_x & avg_share \\
\midrule
0 & 0.0000 & 0.0000 & 0.3848 \\
1 & 0.0024 & -0.0193 & 0.0988 \\
2 & 0.0023 & -0.0260 & 0.0891 \\
3 & 2.0191 & -0.0813 & 0.0430 \\
4 & 1.7516 & -0.1801 & 0.0393 \\
5 & 3.5770 & 1.6926 & 0.1517 \\
6 & 4.4429 & 2.0024 & 0.1932 \\
\bottomrule
\end{tabular}
\end{table}



# 4.3 look at average price and share for products, across market, using $\Gamma$, what is driving differences in prices and market shares

df_all

In [ ]:
import pandas as pd

# 1. Group by product, compute mean for p, x, and shares
table = (
    df_all
    .groupby('choice', as_index=False)
    .agg(
        avg_price=('p','mean'),
        avg_x=('x','mean'),
        avg_share=('shares','mean')
    )
)

# 2. Convert to LaTeX
latex_table = table.to_latex(
    index=False,
    float_format="%.4f",    # set decimal precision as needed
    caption="Average Price, x, and Share by Product",
    label="tab:avg_price_x_share"
)
print(latex_table)


# 4.4 PyBLP

In [36]:
%pip install pyblp
import pyblp


[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
df_blp = df_all.copy()
df_blp = df_blp.rename(columns={'p': 'prices'})
df_blp = df_blp.rename(columns={'choice': 'product_ids'})
df_blp = df_blp.rename(columns={'market': 'market_ids'})



df_blp = df_blp.rename(columns={'z1': 'demand_instruments0'})
df_blp = df_blp.rename(columns={'z2': 'demand_instruments1'})
df_blp = df_blp.rename(columns={'z3': 'demand_instruments2'})
df_blp = df_blp.rename(columns={'z4': 'demand_instruments3'})
df_blp = df_blp.rename(columns={'z5': 'demand_instruments4'})
df_blp = df_blp.rename(columns={'z6': 'demand_instruments5'})


df_blp = df_blp[df_blp['product_ids'] != 0]

# X1 are things that have homog effects
#X2 are (potentially same things) with heterog. effects
X1_formulation = pyblp.Formulation('0+ prices + x') # no constant?
X2_formulation = pyblp.Formulation('0 + prices + x') # random coefficient for outside good.? not in this model
product_formulations = (X1_formulation,X2_formulation)
product_formulations



(prices + x, prices + x)

In [86]:
df_blp.product_ids.unique()

array([1, 2, 3, 4, 5, 6])

In [87]:
df_blp[df_blp['market_ids'] ==1]

,market_ids,product_ids,shares,prices,x,demand_instruments0,demand_instruments1,demand_instruments2,demand_instruments3,demand_instruments4,demand_instruments5
1,1,1,0.0099,0.000278,-0.210973,-6.363361,-5.509449,-5.371022,-1.583637,-1.937069,-0.341810
2,1,2,0.2085,0.000457,-1.557577,-5.198161,-4.396036,-4.265266,-2.862346,-3.498094,-2.330125
3,1,3,0.0090,1.016838,-0.799984,-0.040836,0.835758,-0.520122,-0.797271,0.768238,-0.973296
4,1,4,0.0348,0.109472,-1.338366,-1.538905,-1.172700,0.189448,-1.798173,1.519424,-0.264684
5,1,5,0.3661,5.387786,3.316330,0.758555,-0.700470,0.723864,-0.370940,-1.288456,0.080449
6,1,6,0.0231,4.188119,2.154527,1.044639,-0.724283,-1.325333,-1.520450,-0.922602,1.716677


In [88]:
mc_integration = pyblp.Integration('monte_carlo', size=50, specification_options={'seed': 0})
mc_integration

Configured to construct nodes and weights with Monte Carlo simulation with options {seed: 0}.

In [89]:
mc_problem = pyblp.Problem(product_formulations, df_blp, integration=mc_integration)
mc_problem

Initializing the problem ...
Initialized the problem after 00:00:00.

Dimensions:
 T    N    I     K1    K2    MD 
---  ---  ----  ----  ----  ----
100  600  5000   2     2     7  

Formulations:
       Column Indices:           0      1 
-----------------------------  ------  ---
 X1: Linear Characteristics    prices   x 
X2: Nonlinear Characteristics  prices   x 


Dimensions:
 T    N    I     K1    K2    MD 
---  ---  ----  ----  ----  ----
100  600  5000   2     2     7  

Formulations:
       Column Indices:           0      1 
-----------------------------  ------  ---
 X1: Linear Characteristics    prices   x 
X2: Nonlinear Characteristics  prices   x 

In [90]:
bfgs = pyblp.Optimization('bfgs', {'gtol': 1e-4})


In [92]:
# need to restrict parameters to be lower triangular. I think. Unclear. There's some documentation here talking about Cholesky decompositions.
# Does seem like they ahve the taste shocks baked in here to some degree 'It is common to assume that 𝑓(𝛽𝑖∣𝜃) follows a multivariate normal distribution, and to break it up into three parts: 

# Might need to do something for beta too. 
results = mc_problem.solve(sigma= np.ones((2,2)), optimization=bfgs)
results

Solving the problem ...

Nonlinear Coefficient Initial Values:
Sigma:     prices            x        |  Sigma Squared:     prices            x      
------  -------------  -------------  |  --------------  -------------  -------------
prices  +1.000000E+00                 |      prices      +1.000000E+00  +1.000000E+00
  x     +1.000000E+00  +1.000000E+00  |        x         +1.000000E+00  +2.000000E+00
Starting optimization ...

GMM   Computation  Optimization   Objective   Fixed Point  Contraction  Clipped    Objective      Objective      Gradient                                                
Step     Time       Iterations   Evaluations  Iterations   Evaluations  Shares       Value       Improvement       Norm                          Theta                   
----  -----------  ------------  -----------  -----------  -----------  -------  -------------  -------------  -------------  -------------------------------------------
 1     00:00:00         0             1          1388   

Problem Results Summary:
GMM     Objective      Gradient         Hessian         Hessian     Clipped  Weighting Matrix  Covariance Matrix
Step      Value          Norm       Min Eigenvalue  Max Eigenvalue  Shares   Condition Number  Condition Number 
----  -------------  -------------  --------------  --------------  -------  ----------------  -----------------
 2    +1.614963E+02  +1.340816E-05  +5.220724E+01   +1.588827E+02      0      +1.724003E+01      +7.667105E+06  

Cumulative Statistics:
Computation  Optimizer  Optimization   Objective   Fixed Point  Contraction
   Time      Converged   Iterations   Evaluations  Iterations   Evaluations
-----------  ---------  ------------  -----------  -----------  -----------
 00:00:04       Yes          19           33          25688        79771   

Nonlinear Coefficient Estimates (Robust SEs in Parentheses):
Sigma:      prices              x         |  Sigma Squared:      prices              x       
------  ---------------  --------------

In [ ]:
np.ones

<function numpy.ones(shape, dtype=None, order='C', *, device=None, like=None)>